### User OGS games dump

This notebook generates a dump of your OGS games. The OGS API allows easy access to JSON representations of game records. (The JSON API is lighter on the server than dumping the SGFs directly.) An external tool is called to convert these JSON representations into .sgf format.

In [ ]:
import requests
import json
import time
import random
import pandas as pd
import os
from sys import exit

Enter your OAuth keys and user login data here.

In [ ]:
# keys
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"

# used to authorize access to account-specific API calls
username = "YOUR_OGS_USERNAME"
password = "YOUR_OGS_PASSWORD"

The next cell reads in the list of games to be pulled from the server.
If you wish to pull in games from a different table of game data, modify the filename of the .json file accordingly.
An admissible table needs to keep the `'related'` column from raw_stats.

In [ ]:
# gamesList = [x['detail'] for x in pd.read_json('raw_stats.json')['related'].tolist()]
gamesList = [x['detail'] for x in pd.read_json('clean_stats.json')['related'].tolist()]

In [ ]:
# Obtaining authorization token
s = requests.Session()
response = s.post('https://online-go.com/oauth2/token/',data={"grant_type":"password","username":username,"password":password,"client_id":client_id,"client_secret":client_secret})
result = response.json()
token = result['access_token']
headers = {'Authorization': 'Bearer {}'.format(token)}

The following cell fetches the JSON representations of the games and stores them in the `./ogsjson` directory. The OGS API has a rate limit on requests, so there is also a very primitive rate backoff implemented. You will see some hangups in the output as the rate limit is encountered and the loop has to wait out its backoff time. This code block could take several minutes depending on how many games you've played.

In [ ]:
try:
    os.mkdir("ogsjson")
except FileExistsError:
    pass

attempts = 0
max_attempts = 10
#i = 0 # if you want to only pull a certain number of games

for x in gamesList:
#    i += 1
#    if i > 100:
#        break
    endpoint = "https://online-go.com/api/v1/games/" + x[14:]
    response = s.get(endpoint,  headers = headers)
    # If not rate limited, continue
    # If rate limited, wait
    while response.status_code == 429:
        # If rate limited, wait and then start again
        print("Rate limit reached. Backing off before retrying query. Please wait...")
        attempts += 1
        if attempts >= max_attempts:
            print("Too many attempts.")
            break
        else:
            time.sleep((2 ** attempts) + random.random())
            response = s.get(endpoint,  headers = headers)
    # If not rate limited, continue
    attempts = 0
    try:
        f = open("./ogsjson/"+str(response.json()['id'])+".json", 'w', encoding="utf-8")
        f.write(json.dumps(response.json()['gamedata'], indent = 2))
        f.close()
    except KeyError:
        print(x)
        print(response.json())
        exit("KeyError")

This cell uses [ogstosgf](https://github.com/lightvector/ogstosgf) to convert each game to SGF format, then moves the SGFs to an output directory `./ogssgf`.

In [ ]:
%run -i "ogstosgf.py" ogsjson/

try:
    os.mkdir("ogssgf")
except FileExistsError:
    pass

for filename in os.listdir("./ogsjson"):
    if filename.endswith('.sgf'):
        os.replace("./ogsjson/" + filename, "./ogssgf/" + filename)